In [1]:
import numpy as np 
import pandas as pd 
from sparklines  import sparklines 

In [2]:
df = pd.read_csv('data/2018年销售汇总表.csv',parse_dates = ['日期'])

In [3]:
df.head()

,账号,姓名,单品,数量,单价,金额,日期
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2018-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2018-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2018-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2018-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2018-01-01 23:26:55


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
账号    1500 non-null int64
姓名    1500 non-null object
单品    1500 non-null object
数量    1500 non-null int64
单价    1500 non-null float64
金额    1500 non-null float64
日期    1500 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 82.2+ KB


按客人姓名查看消费金额的平均值与总金额

In [5]:
df.groupby('姓名')['金额'].agg(['mean','sum'])

,mean,sum
姓名,,
Barton LLC,1334.615854,109438.50
"Cronin, Oberbrunner and Spencer",1339.321642,89734.55
"Frami, Hills and Schmidt",1438.466528,103569.59
"Fritsch, Russel and Anderson",1385.366790,112214.71
"Halvorson, Crona and Champlin",1206.971724,70004.36
Herman LLC,1336.532258,82865.00
Jerde-Hilpert,1265.072247,112591.43
"Kassulke, Ondricka and Metz",1350.797969,86451.07
Keeling LLC,1363.977027,100934.30


使用 $ 符号格式化字符串，包括2位小数点

In [6]:
(
    df.groupby('姓名')['金额']
    .agg(['mean', 'sum'])
    .style.format('${0:,.2f}')
)

,mean,sum
姓名,,
Barton LLC,"$1,334.62","$109,438.50"
"Cronin, Oberbrunner and Spencer","$1,339.32","$89,734.55"
"Frami, Hills and Schmidt","$1,438.47","$103,569.59"
"Fritsch, Russel and Anderson","$1,385.37","$112,214.71"
"Halvorson, Crona and Champlin","$1,206.97","$70,004.36"
Herman LLC,"$1,336.53","$82,865.00"
Jerde-Hilpert,"$1,265.07","$112,591.43"
"Kassulke, Ondricka and Metz","$1,350.80","$86,451.07"
Keeling LLC,"$1,363.98","$100,934.30"


金额显示为不带小数点

In [7]:
(
    df.groupby('姓名')['金额']
    .agg(['mean', 'sum'])
    .style.format('${0:,.0f}')
)

,mean,sum
姓名,,
Barton LLC,"$1,335","$109,438"
"Cronin, Oberbrunner and Spencer","$1,339","$89,735"
"Frami, Hills and Schmidt","$1,438","$103,570"
"Fritsch, Russel and Anderson","$1,385","$112,215"
"Halvorson, Crona and Champlin","$1,207","$70,004"
Herman LLC,"$1,337","$82,865"
Jerde-Hilpert,"$1,265","$112,591"
"Kassulke, Ondricka and Metz","$1,351","$86,451"
Keeling LLC,"$1,364","$100,934"


按月分析销售金额，并显示每月销售额占年度销售总额比例

In [8]:
monthly_sales = df.groupby([pd.Grouper(key='日期', freq='M')])['金额'].agg(['sum']).reset_index()
monthly_sales['月占比'] = monthly_sales['sum']/df['金额'].sum()
monthly_sales

,日期,sum,月占比
0,2018-01-31,185361.66,0.091818
1,2018-02-28,146211.62,0.072426
2,2018-03-31,203921.38,0.101012
3,2018-04-30,174574.11,0.086475
4,2018-05-31,165418.55,0.081940
5,2018-06-30,174089.33,0.086235
6,2018-07-31,191662.11,0.094939
7,2018-08-31,153778.59,0.076174
8,2018-09-30,168443.17,0.083438
9,2018-10-31,171495.32,0.084950


使用格式字典控制每列的格式

In [9]:
format_dict = {'sum': '${0:,.0f}', '日期': '{:%Y-%m}', '月占比': '{:.2%}'}
monthly_sales.style.format(format_dict).hide_index()

日期,sum,月占比
2018-01,"$185,362",9.18%
2018-02,"$146,212",7.24%
2018-03,"$203,921",10.10%
2018-04,"$174,574",8.65%
2018-05,"$165,419",8.19%
2018-06,"$174,089",8.62%
2018-07,"$191,662",9.49%
2018-08,"$153,779",7.62%
2018-09,"$168,443",8.34%
2018-10,"$171,495",8.49%


条件格式

In [10]:
(
    monthly_sales
    .style
    .format(format_dict)
    .hide_index()
    .highlight_max(color='lightgreen')
    .highlight_min(color='red')
)

日期,sum,月占比
2018-01,"$185,362",9.18%
2018-02,"$146,212",7.24%
2018-03,"$203,921",10.10%
2018-04,"$174,574",8.65%
2018-05,"$165,419",8.19%
2018-06,"$174,089",8.62%
2018-07,"$191,662",9.49%
2018-08,"$153,779",7.62%
2018-09,"$168,443",8.34%
2018-10,"$171,495",8.49%


渐变色样式

In [11]:
(
    monthly_sales.style
    .format(format_dict)
    .background_gradient(subset=['sum'], cmap='PRGn')
)

,日期,sum,月占比
0,2018-01,"$185,362",9.18%
1,2018-02,"$146,212",7.24%
2,2018-03,"$203,921",10.10%
3,2018-04,"$174,574",8.65%
4,2018-05,"$165,419",8.19%
5,2018-06,"$174,089",8.62%
6,2018-07,"$191,662",9.49%
7,2018-08,"$153,779",7.62%
8,2018-09,"$168,443",8.34%
9,2018-10,"$171,495",8.49%


单元格柱形条

In [12]:
(
    monthly_sales
    .style
    .format(format_dict)
    .hide_index()
    .bar(color='#FFA07A', vmin=100_000, subset=['sum'], align='zero')
    .bar(color='lightblue', vmin=0, subset=['月占比'], align='zero')
    .set_caption('2018年销售一览表')
)

日期,sum,月占比
2018-01,"$185,362",9.18%
2018-02,"$146,212",7.24%
2018-03,"$203,921",10.10%
2018-04,"$174,574",8.65%
2018-05,"$165,419",8.19%
2018-06,"$174,089",8.62%
2018-07,"$191,662",9.49%
2018-08,"$153,779",7.62%
2018-09,"$168,443",8.34%
2018-10,"$171,495",8.49%


使用 Sparkline 实现可视化效果

In [13]:
def sparkline_str(x):
    bins=np.histogram(x)[0]
    sl = ' '.join(sparklines(bins))
    return sl

sparkline_str.__name__ = "走势图"

In [14]:
df.groupby('姓名')['数量', '金额'].agg(['mean', sparkline_str])

数量                       金额  \
                                      mean         走势图         mean   
姓名                                                                    
Barton LLC                       24.890244  ▄▄▃▂▃▆▄█▁▄  1334.615854   
Cronin, Oberbrunner and Spencer  24.970149  █▄▁▄▄▇▅▁▄▄  1339.321642   
Frami, Hills and Schmidt         26.430556  ▄▄▁▂▇█▂▂▅▅  1438.466528   
Fritsch, Russel and Anderson     26.074074  ▁▄▇▃▂▂█▃▄▄  1385.366790   
Halvorson, Crona and Champlin    22.137931  ▇▆▆▇█▁▄▂▄▃  1206.971724   
Herman LLC                       24.806452  ▄▃▅▁▆▄▂▆▃█  1336.532258   
Jerde-Hilpert                    22.460674  ▄▄█▁▂▅▃▂▄▃  1265.072247   
Kassulke, Ondricka and Metz      25.734375  ▂▂▁▁▂▂▁▅▄█  1350.797969   
Keeling LLC                      24.405405  ▁▄▇▃▅█▃▄▃▆  1363.977027   
Kiehn-Spinka                     22.227848  ▃▂█▂▃▅▄▁▄▁  1260.870506   
Koepp Ltd                        21.829268  ▅▇█▆▄▇▅▁▅▇  1264.152927   
Kuhn-Gusikowski                  22.808219  ▂▄█▄▃▁█▄▂▄  1247.866849   
Kulas Inc                        24.095745  ▇▃▇▂▇▁▅▆█▆  1461.191064   
Pollich LLC                      23.383562  █▂▅▂▃▆▁▂▄▇  1196.536712   
Purdy-Kunde                      27.358491  ▄▅█▁▄▇▅█▇█  1469.777547   
Sanford and Sons                 24.000000  ▂▂▂▆▁█▂▂▁▅  1391.872958   
Stokes LLC                       24.527778  ▄▄▄▁▃▂▄█▄▄  1271.332222   
Trantow-Barrows                  24.159574  ▅▄▅▂▃▃▄▄▁█  1312.567872   
White-Trantow                    26.255814  ▃▂▆▆▃██▁██  1579.558023   
Will LLC                         24.702703  ▂█▁▄▄▄▂▅▅▄  1411.318919   

                                             
                                        走势图  
姓名                                           
Barton LLC                       █▄▃▆▄▄▁▁▁▁  
Cronin, Oberbrunner and Spencer  █▅▅▃▃▃▂▂▁▁  
Frami, Hills and Schmidt         █▅▄▇▅▃▄▁▁▁  
Fritsch, Russel and Anderson     ▇█▃▄▂▂▁▂▁▁  
Halvorson, Crona and Champlin    ██▆▅▁▃▂▂▂▂  
Herman LLC                       █▅▇▄▅▄▁▃▂▂  
Jerde-Hilpert                    █▄▅▂▁▂▃▂▂▁  
Kassulke, Ondricka and Metz      █▆▆▄▄▃▂▁▁▂  
Keeling LLC                      ▅█▆▃▄▂▂▁▁▁  
Kiehn-Spinka                     █▇▄▃▃▂▁▂▁▁  
Koepp Ltd                        █▇▅▂▄▂▂▂▁▁  
Kuhn-Gusikowski                  ▆█▄▃▃▃▃▁▁▁  
Kulas Inc                        █▅█▂▄▄▄▂▁▁  
Pollich LLC                      █▆▆▃▃▃▂▂▁▁  
Purdy-Kunde                      █▅▄▃▄▃▃▂▁▂  
Sanford and Sons                 ▅▇█▃▃▄▁▃▁▂  
Stokes LLC                       █▇▆▆▄▂▂▃▁▂  
Trantow-Barrows                  █▇▅▆▃▂▂▁▁▁  
White-Trantow                    ▄█▆▅▄▃▄▂▁▃  
Will LLC                         ▆█▄▅▄▂▁▂▁▂